In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
import numpy as np
import pandas as pd
import re
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException


In [9]:
url = "https://www.transtats.bts.gov/ONTIME/Departures.aspx"

def obtener_opciones(url, aeropuertos_aerolineas):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    seleccionar_aeropuerto_aerolinea = soup.find("select", attrs={"name": aeropuertos_aerolineas})
    opciones = seleccionar_aeropuerto_aerolinea.find_all("option")
    listado_opciones = [opcion.text for opcion in opciones]
    return listado_opciones

listado_aeropuertos = obtener_opciones(url, "cboAirport")
listado_aerolineas = obtener_opciones(url, "cboAirline")

In [5]:
def seleccionar_por_indice(url, nombre_seleccionado, lista_elementos):
    driver = webdriver.Firefox()
    driver.get(url)

    seleccion = Select(driver.find_element(By.NAME, nombre_seleccionado))
    for elemento in lista_elementos:
        seleccion.select_by_visible_text(elemento)
        sleep(1) 

In [6]:


def preselecciones(driver):
    driver.find_element(By.ID, "chkAllStatistics").click() #click_all_statics
    driver.find_element(By.ID, "chkAllDays").click() #click_all_days
    driver.find_element(By.ID, "chkMonths_11").click() #click_mes_diciembre
    
    # Selecciona 2021, 22 y 23
    driver.find_element(By.ID, "chkYears_34").click()
    driver.find_element(By.ID, "chkYears_35").click() 
    driver.find_element(By.ID, "chkYears_36").click() 

In [9]:
driver = webdriver.Firefox()
driver.get(url)

preselecciones(driver)

for aeropuerto in listado_aeropuertos:
    select_aeropuerto = Select(driver.find_element(By.NAME, "cboAirport"))
    select_aeropuerto.select_by_visible_text(aeropuerto)
    # sleep(1)
    for aerolinea in listado_aerolineas:
        select_aerolinea = Select(driver.find_element(By.NAME, "cboAirline"))
        select_aerolinea.select_by_visible_text(aerolinea)
    
        click_submit = driver.find_element(By.ID, "btnSubmit").click()
        
        # sleep(1)
        #el codigo de abajo hace una flecha hacia abajo,
        driver.execute_script("window.scrollBy(0, 200);")
        try:
            element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, "DL_CSV")))
            # sleep(1)
            element.click()
        except:
            pass

driver.quit()
